<a href="https://colab.research.google.com/github/nathalyAlarconT/GenAI_Workshops/blob/main/Intro_to_RAG_Gemma_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup del Notebook

## Instalando las librerías Requeridas

In [1]:
!pip install --upgrade -q langchain --quiet
!pip install google-generativeai langchain-google-genai --quiet
!pip install chromadb pypdf2 python-dotenv --quiet
!pip install PyPDF --quiet
!pip install -U langchain-community --quiet
!pip install sentence-transformers --quiet
!pip install langchainhub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras>=3
!pip install -q langchain langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.1 MB/s eta 0:00:00


## Importando librerías

In [3]:
from google.colab import userdata
import os
from IPython.display import Markdown

Si la siguiente celda genera un error, reiniciar el notebook

In [4]:
# Load Gemma using LangChain library
from langchain_google_vertexai import GemmaChatLocalKaggle
# If Error on this command, restart the session and run again.

## Configurando credenciales

Si quieres poder loggear y visualizar los prompts, puedes configurar tu API Key de LangSmith/LangChain

In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('SMITH_APIKEY')
GOOGLE_API_KEY = userdata.get('GoogleAIStudio')

Podemos acceder a Gemma 2, desde Kaggle, Hugging Face ó Vertex AI. En este caso usaremos Kaggle.

Recuerda que previamente debes aceptar los términos y condiciones en: https://www.kaggle.com/models/google/gemma-2

Para autenticarnos a Kaggle desde el notebook, puedes descargar tus credenciales (KAGGLE_USERNAME, KAGGLE_KEY) en: https://www.kaggle.com/settings desde la sección API Create New Token

In [6]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")
os.environ["KERAS_BACKEND"] = "jax"  # "jax" Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

## Creación de Folders necesarios

Usaremos dos carpetas:
- Mis Datos, almacenara los PDFs con la info propia
- VectorDB, en esta carpeta se creará la base de datos vectorial

In [7]:
!mkdir /content/MisDatos
!mkdir /content/VectorDB

Adiciona los archivos PDF a la carpeta Mis Datos




## Adicionando PDFs

Ahora descargaremos algunos PDFs a la carpeta MisDatos

In [8]:
!wget -P MisDatos https://sdgs.un.org/sites/default/files/2020-09/SDG%20Resource%20Document_Targets%20Overview.pdf

--2024-09-27 19:02:45--  https://sdgs.un.org/sites/default/files/2020-09/SDG%20Resource%20Document_Targets%20Overview.pdf
Resolving sdgs.un.org (sdgs.un.org)... 34.238.11.122
Connecting to sdgs.un.org (sdgs.un.org)|34.238.11.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423899 (414K) [application/pdf]
Saving to: ‘MisDatos/SDG Resource Document_Targets Overview.pdf’

SDG Resource Docume 100%[===================>] 413.96K   374KB/s    in 1.1s    

2024-09-27 19:02:47 (374 KB/s) - ‘MisDatos/SDG Resource Document_Targets Overview.pdf’ saved [423899/423899]



In [9]:
# !wget -P MisDatos https://www.lostiempos.com/sites/default/files/edicion_online/las_delicias_de_mi_llajta.pdf

In [10]:
# !wget -P MisDatos https://www.volkswagen.com.ar/idhub/content/dam/onehub_pkw/importers/ar/tengo-un-volkswagen/manuales/manuales/2020/Manual%20T-Cross%202020.pdf

También puedes adicionar manualmente archivos a la carpeta Mis Datos

# 1. INDEXING

### Librerías necesarias

In [11]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

In [12]:
# @title Configura el origen de los datos
source_data_folder = "/content/MisDatos" # @param {type:"string"}


## Preparando y formateando el contenido necesario

In [13]:
# Leyendo los PDFs del directorio configurado
loader = PyPDFDirectoryLoader(source_data_folder)
data_on_pdf = loader.load()
# cantidad de páginas cargadas
len(data_on_pdf)

19

In [14]:
# Particionando los datos. Con un tamaño delimitado (chunks) y 200 caracters de overlapping para preservar el contexto
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(data_on_pdf)
# Cantidad de chunks obtenidos
len(splits)

55

In [15]:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
embeddings_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-15-1d3747edcb09>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in yo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Base de datos vectorial

In [16]:
# @title Configura el path a la base de datos
path_db = "/content/VectorDB" # @param {type:"string"}


In [17]:
# Almacenamos los chunks en la base de datos
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model, persist_directory=path_db)

# 2. RETRIEVAL

## Librerías necesarias

In [18]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## Selección / Configuración del LLM

### Using Gemma

In [19]:
# Cargamos Gemma utilizando Langchain

keras_backend: str = "jax"
model_name = "gemma2_instruct_2b_en"
llm = GemmaChatLocalKaggle(
    model_name=model_name,
    model=model_name,
    keras_backend=keras_backend,
)



In [20]:
# Helpers
from langchain_core.output_parsers import BaseTransformOutputParser
class GemmaOutputParser(BaseTransformOutputParser[str]):
    """OutputParser that parses LLM response and extract
    the generated part."""

    @classmethod
    def is_lc_serializable(cls) -> bool:
        """Return whether this class is serializable."""
        return True

    @property
    def _type(self) -> str:
        """Return the output parser type for serialization."""
        return "gemma_2_parser"

    def parse(self, text: str) -> str:
        """Return the input text with no changes."""
        model_start_token = "model\n"
        idx = text.rfind(model_start_token)
        return text[idx + len(model_start_token) :] if idx > -1 else ""


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Configuración del Prompt y Retriever



In [21]:
retriever = vectorstore.as_retriever()

# https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=6467f92b-9dac-5816-964f-8abcfa4e4456
prompt = hub.pull("rlm/rag-prompt")
print(f"Prompt:\n\n{prompt.messages[0].prompt.template}")

Prompt:

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [22]:
rag_chain = (
    # Recuperar documentos relevantes a la pregunta
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # El resultado llenará la variable `{question}`
    # y `{context}`
    | prompt
    # El prompt completo se enviará al LLM
    | llm
    # La respuesta del modelo será parseada por GemmaOutputParser
    | GemmaOutputParser()
)

# Ejecución del RAG

In [23]:
# @title Preguntas al Documento
pregunta = "what is the document about?" # @param {type:"string"}
response = rag_chain.invoke(pregunta )
Markdown(response)



This document outlines the Sustainable Development Goals (SDGs) and their targets, focusing on issues related to poverty reduction, gender equality, and sustainable development. It highlights goals for economic growth, access to resources, and the empowerment of women and girls. 
<end_of_turn>

In [24]:
# @title Preguntas al Documento
pregunta = "What the document says about macroeconomic stability ? " # @param {type:"string"}
response = rag_chain.invoke(pregunta )
Markdown(response)



The document states that Goal 13 aims to enhance global macroeconomic stability, including through policy coordination and policy coherence.  This goal recognizes the importance of stable economic conditions for achieving other development goals.  The document emphasizes the need for coordinated efforts to achieve macroeconomic stability. 
<end_of_turn>

In [25]:
# @title Preguntas al Documento
pregunta = "What goals are related to education?" # @param {type:"string"}
response = rag_chain.invoke(pregunta )
Markdown(response)

The goals related to education are to ensure inclusive and equitable quality education for all, including access to early childhood development, technical and vocational education, and lifelong learning opportunities.  These goals aim to provide quality education for all, regardless of gender, disability, or socioeconomic status.  They also focus on increasing access to education for youth and adults, promoting literacy and numeracy, and ensuring equal opportunities for all. 
<end_of_turn>

In [26]:
# cleanup
# vectorstore.delete_collection()

**Fuentes:**

In [27]:
# https://dev.to/timesurgelabs/how-to-use-googles-gemini-pro-with-langchain-1eje

# https://python.langchain.com/v0.2/docs/tutorials/rag/

# https://smith.langchain.com/o/6467f92b-9dac-5816-964f-8abcfa4e4456/projects/p/d35fb5ce-7bac-4627-858c-621aa689239f?timeModel=%7B%22duration%22%3A%227d%22%7D